In [ ]:
import numpy as np
from typing import Optional, Sequence, List, Callable, Union


# --- Expect these to be in scope from your previous conversions ---
# from your_gsl_bs_port import gsl_bs, predict_gsl_bs
# from your_tensor_port import tensor_prod_model_matrix
# from your_glp_port import glp_model_matrix


def _as_2d(a: np.ndarray) -> np.ndarray:
    a = np.asarray(a)
    if a.ndim == 1:
        a = a[:, None]
    return a


def _one_hot_no_intercept(col: np.ndarray) -> np.ndarray:
    """
    R's model.matrix(~col)[,-1] behavior:
      - if col is numeric -> returns the numeric column (no intercept)
      - if col is categorical -> returns k-1 one-hot dummies (baseline drop)
    """
    col = np.asarray(col).ravel()
    # numeric?
    if np.issubdtype(col.dtype, np.number):
        return col.reshape(-1, 1)
    # categorical/object -> treatment coding (drop first level)
    cats = np.unique(col.astype(str))
    if cats.size <= 1:
        return np.empty((col.shape[0], 0))
    baseline = cats[0]
    keep = cats[1:]
    out = np.zeros((col.shape[0], keep.size), dtype=float)
    for j, c in enumerate(keep):
        out[:, j] = (col.astype(str) == c).astype(float)
    return out


def prodspline(
    x: np.ndarray,
    z: Optional[Union[np.ndarray, "pd.DataFrame"]] = None,
    K: Optional[np.ndarray] = None,
    I: Optional[Sequence[int]] = None,
    xeval: Optional[np.ndarray] = None,
    zeval: Optional[Union[np.ndarray, "pd.DataFrame"]] = None,
    knots: str = "quantiles",                 # {"quantiles","uniform"}
    basis: str = "additive",                  # {"additive","tensor","glp"}
    x_min: Optional[Sequence[float]] = None,
    x_max: Optional[Sequence[float]] = None,
    deriv_index: int = 1,                     # 1-based, like R
    deriv: int = 0,
    *,
    gsl_bs: Callable[..., np.ndarray],
    predict_gsl_bs: Callable[..., np.ndarray],
    tensor_prod_model_matrix: Optional[Callable[[List[np.ndarray]], np.ndarray]] = None,
    glp_model_matrix: Optional[Callable[[List[np.ndarray]], np.ndarray]] = None,
) -> np.ndarray:
    """
    Python port of R's `prodspline`.

    Parameters
    ----------
    x : (n, p) array of continuous predictors
    z : optional array/DataFrame of discrete or numeric extra predictors
    K : (p, 2) int array with columns [degree, segments]
        (nbreak used below = segments + 1)
    I : sequence of 0/1 flags per z-column indicating inclusion
    xeval, zeval : evaluation data for x and z (if None, uses training x (and z))
    knots : {"quantiles","uniform"}
    basis : {"additive","tensor","glp"}
    x_min, x_max : optional per-dimension bounds for B-spline construction
    deriv_index : 1-based index of x-dimension for derivative
    deriv : derivative order (0 => function, >0 => derivative)

    You must pass callables:
      - gsl_bs(x_col, degree, nbreak, deriv=0, x_min=None, x_max=None, intercept=False, knots=None) -> ndarray
      - predict_gsl_bs(obj, newx_col) -> ndarray
      - tensor_prod_model_matrix(list_of_designs) -> ndarray   (if basis="tensor")
      - glp_model_matrix(list_of_designs) -> ndarray           (if basis="glp")
    """
    basis = {"additive", "tensor", "glp"}.__contains__(basis) and basis or (_ for _ in ()).throw(
        ValueError("basis must be 'additive', 'tensor', or 'glp'")
    )
    knots = {"quantiles", "uniform"}.__contains__(knots) and knots or (_ for _ in ()).throw(
        ValueError("knots must be 'quantiles' or 'uniform'")
    )
    if K is None:
        raise ValueError("must provide K")
    x = _as_2d(np.asarray(x, dtype=float))
    K = np.asarray(K, dtype=int)
    n, num_x = x.shape
    if K.ndim != 2 or K.shape[0] != num_x or K.shape[1] != 2:
        raise ValueError(f"K must be a (num_x, 2) integer array; got {K.shape}")

    if deriv < 0:
        raise ValueError("deriv is invalid (<0)")
    if deriv_index < 1 or deriv_index > num_x:
        raise ValueError("deriv.index is invalid")
    # In R they warn (not stop) if deriv > degree at deriv_index. We'll just allow it.

    # Determine evaluation data for x
    if xeval is None:
        xeval = x
    else:
        xeval = _as_2d(np.asarray(xeval, dtype=float))
        if xeval.shape[1] != num_x:
            raise ValueError("xeval must have same number of columns as x")

    # Optional z + I handling (can be numpy or pandas)
    if z is not None:
        try:
            import pandas as pd  # for safe column extraction if DataFrame
        except Exception:
            pd = None
        if pd is not None and isinstance(z, pd.DataFrame):
            z_mat = z.values
        else:
            z_mat = np.asarray(z)
            z_mat = _as_2d(z_mat)
        num_z = z_mat.shape[1]
        if I is None:
            raise ValueError("must provide I when z is provided")
        I = np.asarray(I, dtype=int).ravel()
        if I.size != num_z:
            raise ValueError("dimension of z and I incompatible")
        if zeval is not None:
            if pd is not None and isinstance(zeval, pd.DataFrame):
                zeval_mat = zeval.values
            else:
                zeval_mat = np.asarray(zeval)
                zeval_mat = _as_2d(zeval_mat)
            if zeval_mat.shape[1] != num_z:
                raise ValueError("zeval must have the same number of columns as z")
        else:
            zeval_mat = None
    else:
        z_mat = None
        zeval_mat = None
        I = None

    # Decide intercept behavior for gsl_bs:
    # additive and glp use intercept=False; tensor uses intercept=True
    gsl_intercept = (basis == "tensor")

    # Prepare x_min/x_max vectors (optional)
    x_min_vec = None if x_min is None else np.asarray(x_min, dtype=float).ravel()
    x_max_vec = None if x_max is None else np.asarray(x_max, dtype=float).ravel()
    if x_min_vec is not None and x_min_vec.size != num_x:
        raise ValueError("x_min must have length equal to number of x columns")
    if x_max_vec is not None and x_max_vec.size != num_x:
        raise ValueError("x_max must have length equal to number of x columns")

    # ---- build per-variable bases for continuous x (list tp) ----
    tp: List[np.ndarray] = []
    # 1-based to 0-based deriv index
    di0 = deriv_index - 1

    for i in range(num_x):
        deg_i, seg_i = int(K[i, 0]), int(K[i, 1])
        if deg_i <= 0:
            continue  # skip variables with degree 0
        # nbreak = segments + 1
        nbreak = seg_i + 1

        # knots vector
        if knots == "uniform":
            knots_vec = None
        else:
            # quantile knots from training x[:, i]
            probs = np.linspace(0.0, 1.0, num=nbreak)
            xi = x[:, i]
            knots_vec = np.percentile(xi, probs * 100.0)
            # jitter to avoid repeated knots (like R's 1e-10 * range trick)
            rng = float(np.max(xi) - np.min(xi))
            if rng > 0:
                knots_vec = knots_vec + np.linspace(0.0, 1e-10 * rng, num=len(knots_vec))

        # bounds for this coordinate
        x_min_i = None if x_min_vec is None else x_min_vec[i]
        x_max_i = None if x_max_vec is None else x_max_vec[i]

        # Build object on training x, then predict at xeval (to match R's flow)
        if (i == di0) and (deriv != 0):
            obj = gsl_bs(
                x[:, [i]],
                degree=deg_i,
                nbreak=nbreak,
                knots=knots_vec,
                deriv=deriv,
                x_min=x_min_i,
                x_max=x_max_i,
                intercept=gsl_intercept,
            )
        else:
            obj = gsl_bs(
                x[:, [i]],
                degree=deg_i,
                nbreak=nbreak,
                knots=knots_vec,
                deriv=0,
                x_min=x_min_i,
                x_max=x_max_i,
                intercept=gsl_intercept,
            )
        Bi = predict_gsl_bs(obj, newx=xeval[:, [i]])
        tp.append(np.asarray(Bi, dtype=float))

    # ---- add discrete/numeric z pieces indicated by I ----
    if z_mat is not None:
        for i in range(z_mat.shape[1]):
            if int(I[i]) == 1:
                col_train = z_mat[:, i]
                col_eval = col_train if zeval_mat is None else zeval_mat[:, i]
                Zi = _one_hot_no_intercept(col_eval)
                tp.append(Zi)

    # ---- assemble final design ----
    if len(tp) == 0:
        # no relevant predictors -> column of ones with n_eval rows
        P = np.ones((xeval.shape[0], 1), dtype=float)
        # attr "dim.P.no.tensor" would be 0
        return P

    if len(tp) == 1:
        P = tp[0]
        dim_P_no_tensor = P.shape[1]
    else:
        # additive cbind of all blocks
        P = tp[0]
        for i in range(1, len(tp)):
            P = np.column_stack([P, tp[i]])
        dim_P_no_tensor = P.shape[1]
        # tensor or glp if requested
        if basis == "tensor":
            if tensor_prod_model_matrix is None:
                raise ValueError("tensor_prod_model_matrix callable must be provided for basis='tensor'.")
            P = tensor_prod_model_matrix(tp)
        elif basis == "glp":
            if glp_model_matrix is None:
                raise ValueError("glp_model_matrix callable must be provided for basis='glp'.")
            P = glp_model_matrix(tp)
            # GLP derivative masking: zero out columns per original R logic
            if deriv != 0:
                # Build a "deriv marker" list: zeros everywhere, NaNs in the block of the deriv variable
                P_deriv: List[np.ndarray] = []
                for block in tp:
                    P_deriv.append(np.zeros((1, block.shape[1])))
                # Map deriv_index from original x-columns to the tp index:
                # tp only includes x-columns with degree>0 (and z columns with I==1).
                # R code adjusted deriv.index by subtracting count of zero-degree x before it.
                zero_deg_before = np.sum((K[:deriv_index, 0] == 0))  # K is 0-based; deriv_index is 1-based
                tp_idx = deriv_index - 1 - zero_deg_before
                while tp_idx < 0:
                    tp_idx += 1  # keep it valid (rare edge fix)
                tp_idx = int(min(max(tp_idx, 0), len(tp) - 1))
                P_deriv[tp_idx][:] = np.nan
                mask_mat = glp_model_matrix(P_deriv)  # expects to propagate NaNs into columns that involve the deriv var
                mask_vec = np.asarray(mask_mat).ravel()
                not_nan = ~np.isnan(mask_vec)
                # Mirror R: P[, !is.na(glp_model_matrix(P_deriv))] <- 0
                P[:, not_nan] = 0.0

    # You can stash dim_P_no_tensor somewhere if you need it later.
    return np.asarray(P, dtype=float)


In [ ]:
import numpy as np
from typing import Iterable, Optional, Sequence


def is_fullrank(x: np.ndarray) -> bool:
    """
    R's is.fullrank:
      e <- eigen(crossprod(as.matrix(x)), symmetric=TRUE, only.values=TRUE)$values
      e[1] > 0 && abs(e[length(e)]/e[1]) > max(dim(x))*max(sqrt(abs(e)))*.Machine$double.eps
    """
    X = np.asarray(x, dtype=float)
    if X.ndim == 1:
        X = X[:, None]
    XtX = X.T @ X
    # Symmetric eigenvalues; eigh returns ascending order
    e = np.linalg.eigvalsh(XtX)
    e1 = float(e[-1])          # largest
    emin = float(e[0])         # smallest
    # Octave/Matlab-compatible tolerance
    tol = max(X.shape) * float(np.sqrt(np.abs(e)).max(initial=0.0)) * np.finfo(float).eps
    # Full rank if top eig > 0 and (|emin/e1| > tol)
    return (e1 > 0.0) and (abs(emin / e1) > tol)


def RSQfunc(y: Iterable[float], y_pred: Iterable[float], weights: Optional[Iterable[float]] = None) -> float:
    """
    R's RSQfunc:
      if(weights) y <- y*sqrt(w); y_pred <- y_pred*sqrt(w)
      r^2 via centered cross product: [sum((y-ybar)(yhat-ybar))]^2 / (sum((y-ybar)^2) * sum((yhat-ybar)^2))
    """
    y = np.asarray(y, dtype=float).ravel()
    yp = np.asarray(y_pred, dtype=float).ravel()
    if weights is not None:
        w = np.asarray(weights, dtype=float).ravel()
        sw = np.sqrt(w)
        y = y * sw
        yp = yp * sw
    ybar = y.mean()
    num = np.sum((y - ybar) * (yp - ybar)) ** 2
    den = np.sum((y - ybar) ** 2) * np.sum((yp - ybar) ** 2)
    if den <= 0:
        return 0.0
    return float(num / den)


def sqrtm2(x: np.ndarray) -> np.ndarray:
    """
    Numerically stable symmetric (PSD) matrix square root:
      eigendecompose, clip neg eigvals at 0, sqrt, reconstruct.
    Mirrors R's:
      x.eig <- eigen(x, symmetric=TRUE)
      lambda <- sqrt(pmax(x.eig$values, 0))
      x.eig$vectors %*% diag(lambda) %*% t(x.eig$vectors)
    """
    A = np.asarray(x, dtype=float)
    A = 0.5 * (A + A.T)  # enforce symmetry
    vals, vecs = np.linalg.eigh(A)
    vals = np.clip(vals, 0.0, None)
    root = vecs @ (np.sqrt(vals)[:, None] * vecs.T)
    return root


def NZD(a: Iterable[float]) -> np.ndarray:
    """
    Avoid division by zero (R's NZD):
      ifelse(a<0, pmin(-eps, a), pmax(eps, a))
    I.e., clamp to at least machine epsilon in magnitude, preserving sign.
    """
    a = np.asarray(a, dtype=float)
    eps = np.finfo(float).eps
    out = np.where(a < 0, np.minimum(-eps, a), np.maximum(eps, a))
    return out


def dimbs(
    basis: str = "additive",
    degree: Optional[Sequence[int]] = None,
    segments: Optional[Sequence[int]] = None,
) -> int:
    """
    Dimension of a multivariate basis *without* constructing it.
    Mirrors the R function exactly.

    Parameters
    ----------
    basis : {"additive","glp","tensor"}
    degree : sequence of nonnegative ints (per covariate)
    segments : sequence of nonnegative ints (per covariate)

    Returns
    -------
    ncol_bs : int
        Number of basis columns (without an implicit intercept).

    Notes
    -----
    - For "additive": sum over j with degree_j>0 of (degree_j + segments_j - 1).
    - For "tensor":   product over j with degree_j>0 of (degree_j + segments_j).
    - For "glp":      uses the `two_dimen` recursion to avoid materializing the GLP basis.
    """
    if basis not in {"additive", "glp", "tensor"}:
        raise ValueError("basis must be 'additive', 'glp', or 'tensor'")
    if degree is None or segments is None:
        raise ValueError("degree and segments must be provided")

    deg = np.asarray(degree, dtype=int).ravel()
    seg = np.asarray(segments, dtype=int).ravel()
    if deg.size != seg.size:
        raise ValueError("degree and segments must have the same length")

    K = np.column_stack([deg, seg])  # like cbind(degree,segments) in R
    ncol_bs = 0

    if basis == "additive":
        if np.any(K[:, 0] > 0):
            rows = K[K[:, 0] != 0, :]
            ncol_bs = int(np.sum(np.sum(rows, axis=1) - 1))

    elif basis =
